In [14]:
# %run 은 앞서 정의한 ipynb 파일에 저장된 메서드들을 다시 불러오는 역할을 수행합니다. 
# %run 이후에는 파일이 위치한 경로를 입력하여 주세요.
%run .../AnnModel1.ipynb

In [4]:
# 메인 메서드 정의
# 메서드의 매개변수는 사전에 값을 지정하여 주도록 합니다. 
def binary_main(epoch_count = 10, mb_size = 10, report = 1, train_ratio = 0.6):
    
    # 함수의 호출관계에 맞춰 하위 메서드들을 배치합니다.  
    # 이전에 사용된 데이터와는 다른 데이터를 사용하기에 그에 맞는 메서드를 정의할 예정입니다. 
    binary_load_dataset()
    
    # 파라미터 초기화 및 학습과 테스트 기능은 이진 판단 신경망에도 동일하게 사용됩니다. 
    init_param()
    train_and_test(epoch_count, mb_size, report, train_ratio)

In [5]:
# 메서드 정의 
# 이전 과정에서 사용된 메서드와 매우 유사합니다. 
# 본 프로그램에서 사용되는 데이터는 문자열 데이터가 존재하지 않기에, 
# 해당 부분들 제외하며 데이터를 변수처리 합니다. 
def binary_load_dataset():

    # 본 메서드의 동작 순서
    # 1)파일 열기  2)파일 읽기  3-1)파일 전처리 및 변수화  3-2)실험용 Data 생성

    # 1) 파일 열기 
    # with open() 메서드를 활용해 csv 데이터를 열어 csvfile 이름으로 저장 합니다. 
    with open('.../binary_classification_data.csv') as csvfile:
        # 2) 파일 읽기 
        # csvfile 변수를 csv.reader() 메서드를 통해 읽어 줍니다. 
        csvreader = csv.reader(csvfile)
        
        # 3-1) 파일 전처리 및 변수화 
        # 본 파일의 경우 첫 번째 행은 변수 이름이기에 이를 건너뛰어 줍니다. 
        next(csvreader)
        
        # append() 메서드를 통해 값을 저장하기 위한 변수 생성 
        rows = []
        # 반복문 및 append() 메서드를 통해 값을 한 줄씩 꺼내와 rows 변수에 저장합니다. 
        for row in csvreader:
            rows.append(row)
    
    # 3-2) 실험용 Data 및 변수 생성 
    # 추후 다양한 메서드에서 활용하기 위해 전역 변수들을 선언합니다. 
    global data, input_cnt, output_cnt

    # input_cnt  : 신경망에 입력되는 변수의 개수
    # output_cnt : 신경망을 통과해 출력되어지는 변수의 개수  
    # data       : 실험에 쓰일 데이터 
    input_cnt, output_cnt = 8, 1 
    data = np.asarray(rows, dtype = 'float32')


In [6]:
# 메서드 정의 
# 기존 회귀 모델에서 사용되어지는 손실 함수 메서드를 
# 이진 판단 모델에 맞게 정의하는 단계 입니다. 

# 매개변수
# y_hat  : 신경망의 선형연산 결괏값
# y_real : 실제 정답
def forward_postproc(y_hat, y_real):

    # 손실함수를 정의하기 위해서는 시그모이드 교차 엔트로피를 구해줘야 하며,
    # 파이프라인에 맞춰 메서드를 배치시켜 줍니다. 
    entropy  = sigmoid_cross_entropy(y_real,y_hat)
    
    # 미니배치를 고려하게되면 데이터는 총 10개씩 넘어오게 됩니다. 
    # 이를 1 Epoch 에 따른 하나의 손실값으로 정의해야 하기에 평균을 적용합니다. 
    loss = np.mean(entropy)

    # 반환되어지는 두 번째 객체는 
    return loss, [y_real, y_hat, entropy]

In [7]:
# 메서드 정의 
# 손실함수를 정의하기 위해 필요한 시그모이드 교차 엔트로피 수식을 
# 그대로 코드로 옮겨줍니다. 

# 매개변수 
# y_real -> z : 실제 정답
# y_hat  -> x : 신경망의 선형연산 결괏값
def sigmoid_cross_entropy(z,x):
    
    return relu(x) - x * z + np.log(1+np.exp(-np.abs(x)))

In [8]:
# 메서드 정의 
# 활성화 함수 relu 를 코드로 구현합니다.

# 매개변수 
# x : 신경망의 선형연산 결괏값
def relu(x):
    
    return np.maximum(x,0)

In [9]:
# 메서드 정의 
# 학습에 따른 파라미터 갱신을 직접 수행하는 메서드는 backprop_neuralnet() 메서드이며,
# backprop_neuralnet() 메서드에는 G_output 이 필요합니다. 
# G_output는 손실함수의 편미분의 도함수이며, 해당 연산 결괏값은 
def backprop_postproc(aux_pp_real_hat_entropy):
    
    y_real, y_hat, entropy = aux_pp_real_hat_entropy

    # 파라미터를 갱신하는 과정에도 미니배치를 고려할 수 밖에 없으며, 이는 평균과 연결됩니다. 
    # 그렇기에 앞서 회귀 모델의 편미분 과정의 일부를 가져와 적용합니다. 
    # 1 / 미니배치의 크기(M) * 신경망의 출력 크기(N)
    # M과 N은 교차 엔트로피 결괏값의 shape 에서 확보할 수 있습니다. 
    g_loss_entropy = 1.0/np.prod(entropy.shape)
    
    # 시그모이드 교차 엔트로피의 편미분 과정을 메서드로 정의하여 배치 시켜줍니다. 
    g_entropy_output = sigmoid_cross_entropy_with_derv(y_real,y_hat)
    
    # 위 두 값을 곱하여 G_output 을 연산 및 반환시켜 줍니다.  
    G_output = g_entropy_output * g_loss_entropy 
    
    return G_output

In [10]:
# 메서드 정의 
# 시그모이드 교차 엔트로피의 편미분 수식을 그대로 코드로 옮겨와 줍니다. 
# 매개변수 
# z : 실제 정답
# x : 신경망의 선형 연산 결괏값  
def sigmoid_cross_entropy_with_derv(z, x):
    
    return -z + sigmoid(x)

In [11]:
# 메서드 정의 
# 두 가지 한계점을 극복한 시그모이드 메서드를 정의합니다. 
# 매개변수
# x : 신경망의 선형 연산 결괏값
def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))

In [12]:
# 메서드 정의 
# 평가를 수행하는 함수를 정의합니다. 
# 이진 판단의 정확도 여부를 측정하는 방법에는 여러가지가 있지만, 
# 이번에는 신경망이 예측한 이진 판단 결과와 실제 이진 판단의 결과를 비교하여
# 몇 번이나 정답을 맞췄는지를 세어 평균을 내보는 방식을 취하겠습니다. 

# 매개변수 
# y_hat  : 신경망의 선형 연산 결괏값
# y_real : 실제 정답
def eval_accuracy(y_hat,y_real):
    # 신경망이 0 과 1 중 어떻게 판단하였는지는 선형 연산 결괏값의 부호를 조사하면 확인할 수 있습니다. 
    # y_hat이 0을 넘으면 True, 그렇지 못하면 False 를 반환하도록 합니다.  
    estimate = np.greater(y_hat,0)

    # 실제 정답은 0과 1의 값을 갖습니다. 그렇기에 
    # 0.5를 기준으로 True 와 False 를 구분하도록 하겠습니다. 
    answer = np.greater(y_real,0.5)

    # 두 결괏값을 비교하여 얼마나 일치하는지 살펴봅니다. 
    correct = np.equal(estimate, answer)
    
    # 그에 따른 평균을 내어 정답률을 연산합니다. 
    return np.mean(correct)            